<a href="https://colab.research.google.com/github/ROARMarketingConcepts/PySpark-Projects/blob/main/Decision_Trees_Random_Forests_with_PySpark_MLlib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Decision Trees/Random Forests with PySpark MLlib


Initiate Spark session and import packages...

In [ ]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark import SparkConf, SparkContext
from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession \
       .builder \
       .appName("PySpark Book Ch. 4") \
       .getOrCreate()

spark

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,901 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,927 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Package

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_path = '/content/drive/MyDrive/Colab_Notebooks/PySpark Data Analysis/Advanced Analytics With PySpark - OReilly/covertype/'

raw_data = spark.read.csv(file_path+'covtype.data',inferSchema=True)
raw_data.show()

+----+---+---+---+---+----+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
| _c0|_c1|_c2|_c3|_c4| _c5|_c6|_c7|_c8| _c9|_c10|_c11|_c12|_c13|_c14|_c15|_c16|_c17|_c18|_c19|_c20|_c21|_c22|_c23|_c24|_c25|_c26|_c27|_c28|_c29|_c30|_c31|_c32|_c33|_c34|_c35|_c36|_c37|_c38|_c39|_c40|_c41|_c42|_c43|_c44|_c45|_c46|_c47|_c48|_c49|_c50|_c51|_c52|_c53|_c54|
+----+---+---+---+---+----+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|2596| 51|  3|258|  0| 510|221|232|148|6279|   1|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0| 

In [ ]:
raw_data.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- _c1: integer (nullable = true)
 |-- _c2: integer (nullable = true)
 |-- _c3: integer (nullable = true)
 |-- _c4: integer (nullable = true)
 |-- _c5: integer (nullable = true)
 |-- _c6: integer (nullable = true)
 |-- _c7: integer (nullable = true)
 |-- _c8: integer (nullable = true)
 |-- _c9: integer (nullable = true)
 |-- _c10: integer (nullable = true)
 |-- _c11: integer (nullable = true)
 |-- _c12: integer (nullable = true)
 |-- _c13: integer (nullable = true)
 |-- _c14: integer (nullable = true)
 |-- _c15: integer (nullable = true)
 |-- _c16: integer (nullable = true)
 |-- _c17: integer (nullable = true)
 |-- _c18: integer (nullable = true)
 |-- _c19: integer (nullable = true)
 |-- _c20: integer (nullable = true)
 |-- _c21: integer (nullable = true)
 |-- _c22: integer (nullable = true)
 |-- _c23: integer (nullable = true)
 |-- _c24: integer (nullable = true)
 |-- _c25: integer (nullable = true)
 |-- _c26: integer (nullable = true)
 |-- _

Establish column names...

In [ ]:
colnames = ["Elevation", "Aspect", "Slope", \
            "Horizontal_Distance_To_Hydrology", \
            "Vertical_Distance_To_Hydrology", "Horizontal_Distance_To_Roadways", \
            "Hillshade_9am", "Hillshade_Noon", "Hillshade_3pm", \
            "Horizontal_Distance_To_Fire_Points"] + \
            [f"Wilderness_Area_{i}" for i in range(4)] + \
            [f"Soil_Type_{i}" for i in range(40)] + \
            ["Cover_Type"]

colnames

['Elevation',
 'Aspect',
 'Slope',
 'Horizontal_Distance_To_Hydrology',
 'Vertical_Distance_To_Hydrology',
 'Horizontal_Distance_To_Roadways',
 'Hillshade_9am',
 'Hillshade_Noon',
 'Hillshade_3pm',
 'Horizontal_Distance_To_Fire_Points',
 'Wilderness_Area_0',
 'Wilderness_Area_1',
 'Wilderness_Area_2',
 'Wilderness_Area_3',
 'Soil_Type_0',
 'Soil_Type_1',
 'Soil_Type_2',
 'Soil_Type_3',
 'Soil_Type_4',
 'Soil_Type_5',
 'Soil_Type_6',
 'Soil_Type_7',
 'Soil_Type_8',
 'Soil_Type_9',
 'Soil_Type_10',
 'Soil_Type_11',
 'Soil_Type_12',
 'Soil_Type_13',
 'Soil_Type_14',
 'Soil_Type_15',
 'Soil_Type_16',
 'Soil_Type_17',
 'Soil_Type_18',
 'Soil_Type_19',
 'Soil_Type_20',
 'Soil_Type_21',
 'Soil_Type_22',
 'Soil_Type_23',
 'Soil_Type_24',
 'Soil_Type_25',
 'Soil_Type_26',
 'Soil_Type_27',
 'Soil_Type_28',
 'Soil_Type_29',
 'Soil_Type_30',
 'Soil_Type_31',
 'Soil_Type_32',
 'Soil_Type_33',
 'Soil_Type_34',
 'Soil_Type_35',
 'Soil_Type_36',
 'Soil_Type_37',
 'Soil_Type_38',
 'Soil_Type_39',
 'Cov

In [ ]:
data = raw_data.toDF(*colnames).withColumn("Cover_Type",F.col("Cover_Type").cast(T.DoubleType()))
data.printSchema()

root
 |-- Elevation: integer (nullable = true)
 |-- Aspect: integer (nullable = true)
 |-- Slope: integer (nullable = true)
 |-- Horizontal_Distance_To_Hydrology: integer (nullable = true)
 |-- Vertical_Distance_To_Hydrology: integer (nullable = true)
 |-- Horizontal_Distance_To_Roadways: integer (nullable = true)
 |-- Hillshade_9am: integer (nullable = true)
 |-- Hillshade_Noon: integer (nullable = true)
 |-- Hillshade_3pm: integer (nullable = true)
 |-- Horizontal_Distance_To_Fire_Points: integer (nullable = true)
 |-- Wilderness_Area_0: integer (nullable = true)
 |-- Wilderness_Area_1: integer (nullable = true)
 |-- Wilderness_Area_2: integer (nullable = true)
 |-- Wilderness_Area_3: integer (nullable = true)
 |-- Soil_Type_0: integer (nullable = true)
 |-- Soil_Type_1: integer (nullable = true)
 |-- Soil_Type_2: integer (nullable = true)
 |-- Soil_Type_3: integer (nullable = true)
 |-- Soil_Type_4: integer (nullable = true)
 |-- Soil_Type_5: integer (nullable = true)
 |-- Soil_Type

Split data into train and test datasets...

In [ ]:
(train_data, test_data) = data.randomSplit([0.9, 0.1])
train_data.cache()
test_data.cache()

DataFrame[Elevation: int, Aspect: int, Slope: int, Horizontal_Distance_To_Hydrology: int, Vertical_Distance_To_Hydrology: int, Horizontal_Distance_To_Roadways: int, Hillshade_9am: int, Hillshade_Noon: int, Hillshade_3pm: int, Horizontal_Distance_To_Fire_Points: int, Wilderness_Area_0: int, Wilderness_Area_1: int, Wilderness_Area_2: int, Wilderness_Area_3: int, Soil_Type_0: int, Soil_Type_1: int, Soil_Type_2: int, Soil_Type_3: int, Soil_Type_4: int, Soil_Type_5: int, Soil_Type_6: int, Soil_Type_7: int, Soil_Type_8: int, Soil_Type_9: int, Soil_Type_10: int, Soil_Type_11: int, Soil_Type_12: int, Soil_Type_13: int, Soil_Type_14: int, Soil_Type_15: int, Soil_Type_16: int, Soil_Type_17: int, Soil_Type_18: int, Soil_Type_19: int, Soil_Type_20: int, Soil_Type_21: int, Soil_Type_22: int, Soil_Type_23: int, Soil_Type_24: int, Soil_Type_25: int, Soil_Type_26: int, Soil_Type_27: int, Soil_Type_28: int, Soil_Type_29: int, Soil_Type_30: int, Soil_Type_31: int, Soil_Type_32: int, Soil_Type_33: int, S

Create row vectors to comply with the MLlib requirement that all of the inputs be collected into one column, whose value is a vector. The output doesn’t look exactly like a sequence of numbers, but that’s because this shows a raw representation of the vector, represented as a SparseVector instance to save storage.

In [ ]:
from pyspark.ml.feature import VectorAssembler

input_cols = colnames[:-1]
vector_assembler = VectorAssembler(inputCols=input_cols,
                                    outputCol="featureVector")

assembled_train_data = vector_assembler.transform(train_data)

assembled_train_data.select("featureVector").show(truncate = False)

+----------------------------------------------------------------------------------------------------+
|featureVector                                                                                       |
+----------------------------------------------------------------------------------------------------+
|(54,[0,1,2,3,4,5,6,7,8,9,13,15],[1859.0,18.0,12.0,67.0,11.0,90.0,211.0,215.0,139.0,792.0,1.0,1.0])  |
|(54,[0,1,2,3,4,5,6,7,8,9,13,15],[1860.0,18.0,13.0,95.0,15.0,90.0,210.0,213.0,138.0,780.0,1.0,1.0])  |
|(54,[0,1,2,3,4,5,6,7,8,9,13,15],[1861.0,35.0,14.0,60.0,11.0,85.0,218.0,209.0,124.0,832.0,1.0,1.0])  |
|(54,[0,1,2,3,4,5,6,7,8,9,13,15],[1863.0,37.0,17.0,120.0,18.0,90.0,217.0,202.0,115.0,769.0,1.0,1.0]) |
|(54,[0,1,2,3,4,5,6,7,8,9,13,15],[1866.0,23.0,14.0,85.0,16.0,108.0,212.0,210.0,133.0,819.0,1.0,1.0]) |
|(54,[0,1,2,3,4,5,6,7,8,9,13,15],[1867.0,20.0,15.0,108.0,19.0,120.0,208.0,206.0,132.0,808.0,1.0,1.0])|
|(54,[0,1,2,3,4,5,6,7,8,9,13,15],[1868.0,27.0,16.0,67.0,17.0,95.0,212.0,2

In [ ]:
assembled_train_data.show()

+---------+------+-----+--------------------------------+------------------------------+-------------------------------+-------------+--------------+-------------+----------------------------------+-----------------+-----------------+-----------------+-----------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+----------+--------------------+
|Elevation|Aspect|Slope|Horizontal_Distance_To_Hydrology|Vertical_Distance_To_Hydrology|Horizontal_Distance_To_Roadways|Hillshade_9am|Hillshade_Noon|Hillshade_3pm|Horizontal_Distance_To_F

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier

classifier = DecisionTreeClassifier(seed = 1234, labelCol="Cover_Type",
                                    featuresCol="featureVector",
                                    predictionCol="prediction")

model = classifier.fit(assembled_train_data)
print(model.toDebugString)

DecisionTreeClassificationModel: uid=DecisionTreeClassifier_5bbb6db03709, depth=5, numNodes=47, numClasses=8, numFeatures=54
  If (feature 0 <= 3044.5)
   If (feature 0 <= 2559.5)
    If (feature 10 <= 0.5)
     If (feature 0 <= 2446.5)
      If (feature 3 <= 15.0)
       Predict: 4.0
      Else (feature 3 > 15.0)
       Predict: 3.0
     Else (feature 0 > 2446.5)
      If (feature 17 <= 0.5)
       Predict: 2.0
      Else (feature 17 > 0.5)
       Predict: 3.0
    Else (feature 10 > 0.5)
     If (feature 9 <= 5416.5)
      Predict: 2.0
     Else (feature 9 > 5416.5)
      If (feature 5 <= 542.0)
       Predict: 2.0
      Else (feature 5 > 542.0)
       Predict: 5.0
   Else (feature 0 > 2559.5)
    If (feature 0 <= 2954.5)
     If (feature 15 <= 0.5)
      If (feature 17 <= 0.5)
       Predict: 2.0
      Else (feature 17 > 0.5)
       Predict: 3.0
     Else (feature 15 > 0.5)
      Predict: 3.0
    Else (feature 0 > 2954.5)
     If (feature 3 <= 211.0)
      If (feature 36 <= 0.5)
    

In [ ]:
import pandas as pd

pd.DataFrame(model.featureImportances.toArray(),
            index=input_cols, columns=['importance']).\
            sort_values(by="importance", ascending=False)

,importance
Elevation,0.804659
Horizontal_Distance_To_Hydrology,0.043469
Soil_Type_3,0.030876
Wilderness_Area_0,0.030441
Hillshade_Noon,0.025705
Soil_Type_1,0.024217
Soil_Type_31,0.017041
Wilderness_Area_2,0.011657
Soil_Type_22,0.006489
Horizontal_Distance_To_Roadways,0.002470


Make predictions on the training data and compare them to the actual values.

In [ ]:
# Ignore value at probability[0], it's always zero.

predictions = model.transform(assembled_train_data)
predictions.select("Cover_Type", "prediction","probability").show(10, truncate=False)

+----------+----------+----------------------------------------------------------------------------------------------+
|Cover_Type|prediction|probability                                                                                   |
+----------+----------+----------------------------------------------------------------------------------------------+
|3.0       |3.0       |[0.0,0.0,0.0327672889638927,0.633864219667405,0.05094399324965721,0.0,0.28242449811904513,0.0]|
|3.0       |3.0       |[0.0,0.0,0.0327672889638927,0.633864219667405,0.05094399324965721,0.0,0.28242449811904513,0.0]|
|3.0       |3.0       |[0.0,0.0,0.0327672889638927,0.633864219667405,0.05094399324965721,0.0,0.28242449811904513,0.0]|
|6.0       |3.0       |[0.0,0.0,0.0327672889638927,0.633864219667405,0.05094399324965721,0.0,0.28242449811904513,0.0]|
|3.0       |3.0       |[0.0,0.0,0.0327672889638927,0.633864219667405,0.05094399324965721,0.0,0.28242449811904513,0.0]|
|3.0       |3.0       |[0.0,0.0,0.03276728896389

Let's evaluate the model performance...

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol="Cover_Type",
                                        predictionCol="prediction")

evaluator.setMetricName("accuracy").evaluate(predictions),evaluator.setMetricName("f1").evaluate(predictions)

(0.7016050244242847, 0.6841008078836962)

Generate a confusion matrix...

In [ ]:
confusion_matrix = predictions.groupBy("Cover_Type").\
  pivot("prediction", range(1,8)).count().\
  na.fill(0.0).\
  orderBy("Cover_Type")

confusion_matrix.show()

+----------+------+------+-----+---+---+---+----+
|Cover_Type|     1|     2|    3|  4|  5|  6|   7|
+----------+------+------+-----+---+---+---+----+
|       1.0|135914| 51503|  165|  0|  0|  0|3206|
|       2.0| 53873|196169| 4280| 94| 49|  0| 533|
|       3.0|     0|  6001|25557|657|  0|  0|   0|
|       4.0|     0|    25| 1449|994|  0|  0|   0|
|       5.0|     7|  7729|  739|  0| 85|  0|   0|
|       6.0|     0|  6327| 8748|529|  0|  0|   0|
|       7.0| 10086|    19|   55|  0|  0|  0|8252|
+----------+------+------+-----+---+---+---+----+



In [ ]:
from pyspark.sql import DataFrame

def class_probabilities(data):
    total = data.count()
    return data.groupBy("Cover_Type").count().\
    orderBy("Cover_Type").select(F.col("count").cast(T.DoubleType())).\
    withColumn("count_proportion", F.col("count")/total).\
    select("count_proportion").collect()


In [ ]:
train_prior_probabilities = class_probabilities(train_data)
test_prior_probabilities = class_probabilities(test_data)

train_prior_probabilities, test_prior_probabilities

([Row(count_proportion=0.3647640260398245),
  Row(count_proportion=0.48752592989130955),
  Row(count_proportion=0.06159125887829919),
  Row(count_proportion=0.004718523262816775),
  Row(count_proportion=0.01636570467168217),
  Row(count_proportion=0.02983299716085614),
  Row(count_proportion=0.03520156009521169)],
 [Row(count_proportion=0.36317214967136474),
  Row(count_proportion=0.48826056204392154),
  Row(count_proportion=0.061051977849466074),
  Row(count_proportion=0.004813083306018942),
  Row(count_proportion=0.016095364603998828),
  Row(count_proportion=0.030413856159539048),
  Row(count_proportion=0.036193006365690825)])

Convert train and test probabilities to a list

In [ ]:
train_prior_probabilities = [p[0] for p in train_prior_probabilities]
test_prior_probabilities = [p[0] for p in test_prior_probabilities]
train_prior_probabilities, test_prior_probabilities

([0.3647640260398245,
  0.48752592989130955,
  0.06159125887829919,
  0.004718523262816775,
  0.01636570467168217,
  0.02983299716085614,
  0.03520156009521169],
 [0.36317214967136474,
  0.48826056204392154,
  0.061051977849466074,
  0.004813083306018942,
  0.016095364603998828,
  0.030413856159539048,
  0.036193006365690825])

$\sum{(P_{i}|train)(P_{i}|test)}$ for each `Cover_Type` value $i$ to determine accuracy of a random guess.

In [ ]:
print(list(zip(train_prior_probabilities, test_prior_probabilities)))

[(0.3647640260398245, 0.36317214967136474), (0.48752592989130955, 0.48826056204392154), (0.06159125887829919, 0.061051977849466074), (0.004718523262816775, 0.004813083306018942), (0.01636570467168217, 0.016095364603998828), (0.02983299716085614, 0.030413856159539048), (0.03520156009521169, 0.036193006365690825)]


In [ ]:
sum([train_p * cv_p for train_p, cv_p in zip(train_prior_probabilities,
                                              test_prior_probabilities)])

0.3767395975744436

Random guessing achieves 37% accuracy then, which makes 70% seem like a good result after all. But the latter result was achieved with default hyperparameters.




Now, we create a pipeline to automate some processes to perform hyperparameter tuning

In [ ]:
from pyspark.ml import Pipeline

assembler = VectorAssembler(inputCols=input_cols, outputCol="featureVector")
classifier = DecisionTreeClassifier(seed=1234, labelCol="Cover_Type",
                                    featuresCol="featureVector",
                                    predictionCol="prediction")

pipeline = Pipeline(stages=[assembler, classifier])

Pipeline_b99f1c444a26

In [ ]:
from pyspark.ml.tuning import ParamGridBuilder

paramGrid = ParamGridBuilder(). \
  addGrid(classifier.impurity, ["gini", "entropy"]). \
  addGrid(classifier.maxDepth, [1, 20]). \
  addGrid(classifier.maxBins, [40, 300]). \
  addGrid(classifier.minInfoGain, [0.0, 0.05]). \
  build()

multiclassEval = MulticlassClassificationEvaluator(). \
  setLabelCol("Cover_Type"). \
  setPredictionCol("prediction"). \
  setMetricName("accuracy")

In [ ]:
from pyspark.ml.tuning import TrainValidationSplit

validator = TrainValidationSplit(seed=1234,
  estimator=pipeline,
  evaluator=multiclassEval,
  estimatorParamMaps=paramGrid,
  trainRatio=0.9)

validator_model = validator.fit(train_data)

In [36]:
from pprint import pprint

best_model = validator_model.bestModel
pprint(best_model.stages[1].extractParamMap())

{Param(parent='DecisionTreeClassifier_d2ffaba31c55', name='labelCol', doc='label column name.'): 'Cover_Type',
 Param(parent='DecisionTreeClassifier_d2ffaba31c55', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes. Must be in range [0, 30].'): 20,
 Param(parent='DecisionTreeClassifier_d2ffaba31c55', name='minInstancesPerNode', doc='Minimum number of instances each child must have after split. If a split causes the left or right child to have fewer than minInstancesPerNode, the split will be discarded as invalid. Should be >= 1.'): 1,
 Param(parent='DecisionTreeClassifier_d2ffaba31c55', name='leafCol', doc='Leaf indices column name. Predicted leaf index of each instance in each tree by preorder.'): '',
 Param(parent='DecisionTreeClassifier_d2ffaba31c55', name='minInfoGain', doc='Minimum information gain for a split to be considered at a tree node.'): 0.0,
 Param(parent='DecisionTreeClassifier_d2ffaba31c5

In [37]:
metrics = validator_model.validationMetrics
params = validator_model.getEstimatorParamMaps()
metrics_params = list(zip(metrics, params))
metrics_params

[(0.6352581845948267,
  {Param(parent='DecisionTreeClassifier_d2ffaba31c55', name='impurity', doc='Criterion used for information gain calculation (case-insensitive). Supported options: entropy, gini'): 'gini',
   Param(parent='DecisionTreeClassifier_d2ffaba31c55', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes. Must be in range [0, 30].'): 1,
   Param(parent='DecisionTreeClassifier_d2ffaba31c55', name='maxBins', doc='Max number of bins for discretizing continuous features.  Must be >=2 and >= number of categories for any categorical feature.'): 40,
   Param(parent='DecisionTreeClassifier_d2ffaba31c55', name='minInfoGain', doc='Minimum information gain for a split to be considered at a tree node.'): 0.0}),
 (0.6352581845948267,
  {Param(parent='DecisionTreeClassifier_d2ffaba31c55', name='impurity', doc='Criterion used for information gain calculation (case-insensitive). Supported options: entropy, gi

In [38]:
metrics_params.sort(key=lambda x: x[0], reverse=True)
metrics_params

[(0.9104705545480577,
  {Param(parent='DecisionTreeClassifier_d2ffaba31c55', name='impurity', doc='Criterion used for information gain calculation (case-insensitive). Supported options: entropy, gini'): 'entropy',
   Param(parent='DecisionTreeClassifier_d2ffaba31c55', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes. Must be in range [0, 30].'): 20,
   Param(parent='DecisionTreeClassifier_d2ffaba31c55', name='maxBins', doc='Max number of bins for discretizing continuous features.  Must be >=2 and >= number of categories for any categorical feature.'): 40,
   Param(parent='DecisionTreeClassifier_d2ffaba31c55', name='minInfoGain', doc='Minimum information gain for a split to be considered at a tree node.'): 0.0}),
 (0.9097642454901212,
  {Param(parent='DecisionTreeClassifier_d2ffaba31c55', name='impurity', doc='Criterion used for information gain calculation (case-insensitive). Supported options: entropy

In [39]:
metrics.sort(reverse=True)
print(metrics[0])

0.9104705545480577


Evaluate the best model on the test dataset...

In [40]:
multiclassEval.evaluate(best_model.transform(test_data))

0.9112771059395863